In [1]:
import warnings
warnings.filterwarnings('ignore')

# INF391 - Tarea 9
El objetivo de esta tarea es construir un sistema de recomendación basado en contenido usando procesamiento del lenguaje natural.
- El conjunto de datos son las 250 películas mejor ranqueadas de IMDB.
- Las recomendaciones estarán basadas en información como directores, actores, género y descripción de la película.
- Realizar una limpieza de los datos para considerar solo las palabras más relevantes de la descripción de la película.
- Vectorizar cada película y calcular su similaridad con el resto.
- La entrada será el título de una película y la salida debe ser una lista con las 10 más similares (Top-10).
- ¿Cómo cambian las recomendaciones si están basadas únicamente en el título de la película?
- ¿Alguna otra *feature* del conjunto original sería interesante incluir en el análisis?

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/cnunez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cnunez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/cnunez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cnunez/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import pandas as pd
df = pd.read_csv('IMDB_Top250.csv')
df = df[['Title','Genre','Director','Actors','Plot']]

# Desarrollo

## Armado del Corpora

Para comenzar a construir nuestro sistema recomendador, debemos definir el corpora sobre el cual trabajaremos. Para ello, consideraremos los datos de las columnas `'Genre','Director','Actors','Plot'` en un solo texto, formando un **Bag of Word**. Para lograr esto, concatenaremos las columnas ya mencionas:

In [5]:
df["corpora"] = df["Genre"] + " " + df["Director"] +" " +df["Director"] +" " +df["Actors"] +" " +df["Plot"]

In [6]:
df_for_prediction = df[['Title', 'corpora']]

## Procesamiento de texto

El siguiente paso es el procesamiento de texto, en este caso, sobre nuestro corpora. En este desarrollo, buscamos limpiar el texto, aplicar una tokenización y una lemanización, para próximamente trabajar sobre nuestro corpora ya procesado.

In [7]:
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [8]:
def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2)

In [9]:
df_for_prediction['corpora'] = df_for_prediction['corpora'].apply(clean_txt)

## Vectorización

Una vez procesado el corpora, debemos vectorizar el texto, con el objetivo de obtener la matriz con los valores **Tf-Ldf**, para proseguir a calcular la similaridad coseno entre las películas del dataset.

In [10]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_movie = tfidf_vectorizer.fit_transform((df_for_prediction['corpora']))

In [11]:
cosine_sim = cosine_similarity(tfidf_movie)

## TOP-10 function

Finalmente desarrollamos la función para nuestro sistema _Content-based Recommendation_ , en este caso, los parámetros de la función son: un titulo de una película ya existente en el dataset, la matriz con las similaridades coseno entre las películas, y el dataframe con los datos iniciales. El output será una lista con las 10 películas con mayor similaridad al titulo dado.

In [12]:
def top_10_by_title(title, cosine_sim = cosine_sim, df = df): 
    
    index_movie = df[df['Title'] == title]
    if index_movie.index.values:
        
        index = index_movie.index[0]
        score_series = pd.Series(cosine_sim[index]).sort_values(ascending = False)
        top_10_indexes = list(score_series.iloc[1:11].index)

        return df.iloc[top_10_indexes]['Title'].values.tolist()
    else:
        print('Movie does not exist')

## Test

In [13]:
top_10_by_title('The Godfather')

['The Godfather: Part II',
 'Apocalypse Now',
 'The Man Who Shot Liberty Valance',
 'The Grapes of Wrath',
 'On the Waterfront',
 'Guardians of the Galaxy',
 'The Night of the Hunter',
 'Casino',
 'Star Wars: Episode VI - Return of the Jedi',
 'Harvey']

In [14]:
top_10_by_title('Harvey')

['The Nightmare Before Christmas',
 'The Grapes of Wrath',
 'Modern Times',
 'The Godfather',
 'The Man Who Shot Liberty Valance',
 'Indiana Jones and the Last Crusade',
 'The Gold Rush',
 'Ben-Hur',
 '12 Years a Slave',
 'American Beauty']

# Análisis

> ¿Cómo cambian las recomendaciones si están basadas únicamente en el título de la película?

Si consideramos un corpora que solo contenga títulos, el calculo de la matriz **Tf-Ldf** y el de la similaridad coseno serán totalmente distinto, dado que un titulo no incluye la metadata de una pelicula; lo cual generará que el algoritmo entregue recomendaciones en base a la similaridad de textos libres de contexto, los cuales solo seguirán estructuras semánticas básicas, dado que su bag of words será muy simple.

> ¿Alguna otra *feature* del conjunto original sería interesante incluir en el análisis?

Para responder a esto, debemos definir que un *feature* se considerara interesante si agrega semántica al corpora, de tal manera que el bag of words aumente en sus datos.

Consideremos el dataframe original:

In [15]:
df = pd.read_csv('IMDB_Top250.csv')

In [16]:
df.columns

Index(['Unnamed: 0', 'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre',
       'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards',
       'Poster', 'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'tomatoMeter', 'tomatoImage',
       'tomatoRating', 'tomatoReviews', 'tomatoFresh', 'tomatoRotten',
       'tomatoConsensus', 'tomatoUserMeter', 'tomatoUserRating',
       'tomatoUserReviews', 'tomatoURL', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response'],
      dtype='object')

De las columnas ya ingresas, las cuales son `'Title','Genre','Director','Actors','Plot'`, sería interesante agregar las columnas `'Writer','Production','Language','Country','Awards'`, dado que son los que presentan mayor cantidad de palabras con contexto que aportarían a la semántica del bag of words.